In [ ]:
import numpy as np
from qutip import *
from qutip.qip.operations import rx
import scipy as sp
from scipy.optimize import curve_fit,fsolve
from scipy.special import erfc,erf
import os
import sys
import time
from copy import deepcopy
import matplotlib.pyplot as plt
import scipy.constants as sc
import scqubits as scq
e_c=(sc.e)**2/2/sc.h/10**(-6)

#parameters to be changed
EJ1=7.3
EJJ=60/corr#JJA junction energy
E1=0.3032002286433497*corr#correct!
N=int(122/corr)
Charge1=0.9177436958045079
gq1=0.07090222363138766
gp1=0.0026783764551955858*2
gpq1=0.27734680023872377*2

"""
Will Oliver's params
EJ1=6.2
EJJ=81.6#JJA junction energy
E1=0.3686901660093396#correct!
N=102
Charge1=1.2644130824354534
gq1=0.0976848978022153
gp1=0.006860267400351946
gpq1=0.710383041405096
"""


omega1=np.sqrt(8*E1*EJJ)
EL=EJJ/N
print(EL,omega1)#should be constant (0.5,12.06)
zpf_pp1=((EJJ/8/E1)**(1/4))/np.sqrt(2)
zpf_p=((EL/8/Charge1)**(1/4))/np.sqrt(2)
zpf_x=1/(2*zpf_p)


#Fock Basis
N1=100
a=destroy(N1)
ad=a.dag()
nOp1=-1j*(a-ad)*zpf_p
fOp1=(a+ad)*zpf_x
cosfOp1=((1j*fOp1).expm()+(-1j*fOp1).expm())/2
H0_qub= 4*Charge1*(nOp1**2)+(EJ1)*cosfOp1+0.5*(EL)*(fOp1)**2
qub_energies,qub_states=H0_qub.eigenstates()

#Fluxonium basis
N1=30
H0_new=Qobj(np.matrix([[(np.matrix(qub_states[j].dag())*np.matrix(H0_qub)*np.matrix(qub_states[i]))[0,0] for i in range(N1)] for j in range(N1)]))
nOp_new=Qobj(np.matrix([[(np.matrix(qub_states[j].dag())*np.matrix(nOp1)*np.matrix(qub_states[i]))[0,0] for i in range(N1)] for j in range(N1)]))
fOp_new=Qobj(np.matrix([[(np.matrix(qub_states[j].dag())*np.matrix(fOp1)*np.matrix(qub_states[i]))[0,0] for i in range(N1)] for j in range(N1)]))
qub_energies1,qub_states1=H0_new.eigenstates()
n_new=0*qeye(N1)
for i in range(N1):
    n_new=n_new+i*ket2dm(qub_states1[i])
    
print(qub_energies1)
print([np.abs(expect(n_new,state)) for state in qub_states1])

#parasitic mode
N2=50#truncation error affects the commutation relations!
pa=destroy(N2)
pad=pa.dag()
pn_old=-1j*zpf_pp1*(pa-pad)
pf_old=1/2/zpf_pp1*(pa+pad)
H1_old= 4*E1*(pn_old)**2+0.5*(EJJ)*(pf_old)**2 
par_energies,par_states=H1_old.eigenstates()
print(np.diff(par_energies)[:2])
N2=10
H1_par=Qobj(np.matrix([[(np.matrix(par_states[j].dag())*np.matrix(H1_old)*np.matrix(par_states[i]))[0,0] for i in range(N2)] for j in range(N2)]))
pn=Qobj(np.matrix([[(np.matrix(par_states[j].dag())*np.matrix(pn_old)*np.matrix(par_states[i]))[0,0] for i in range(N2)] for j in range(N2)]))
par_energies,par_states=H1_par.eigenstates()

H1=tensor(H0_new,qeye(N2))+tensor(qeye(N1),H1_par)
Hint=gpq1*tensor(nOp_new,pn)#interaction between qubit and the parasitic modes
num_p=tensor(qeye(N1),H1_par/omega1-0.5*qeye(N2))
num_t=tensor(n_new,qeye(N2))
omega_d=np.linspace(8.5,9.5,1000)
nr=np.linspace(0,50,51)
flux_file=[]
par_file=[]
cross_file=[]
for i in range(3):
    en_i=[]
    en_j=[]
    pq_ex=[]
    pp_ex=[]
    en_ex=[]
    t=0
    print(i)
    for omegad in omega_d:
        T = 2*np.pi/omegad
        args={'w':omegad}
        psi_old=tensor(qub_states1[i],par_states[0])
        q_ex = np.zeros(len(nr))
        p_ex = np.zeros(len(nr))
        e_ex = np.zeros(len(nr))
        over_ex=np.zeros(len(nr))
        for n,nbar in enumerate(nr):
            Ht=np.sqrt(nbar)*(2*gp1*tensor(qeye(N1),pn)+2*gq1*tensor(nOp_new,qeye(N2)))
            H = [H1+Hint, [Ht, lambda t,args: np.cos(args['w'] * t)]]#2\pi f t
            f_modes_0,f_energies=floquet_modes(H,T, args)
            arg_max=np.argmax([np.abs(psi_old.overlap(f_modes_0[i])) for i in range(N1*N2)])
            psi_t=f_modes_0[arg_max]
            psi_old=psi_t
            q_ex[n] = np.abs(expect(num_t, psi_t))
            p_ex[n] = np.abs(expect(num_p, psi_t))
            e_ex[n] = np.abs(np.mod(f_energies[arg_max],args['w']))#the quasienergies should be using omega and not f!
        pq_ex.append(q_ex)
        
        en_ex.append(e_ex)
        pp_ex.append(p_ex)
    flux_file.append(pq_ex)
    par_file.append(pp_ex)
    cross_file.append(en_ex)

0.4918032786885246 12.06383478620326
[-1.53894102 -1.50875833  4.53580038  5.1255285   9.0636406  11.32333323
 14.38239149 15.96329457 16.14595676 17.34925111 19.55831369 21.12251505
 22.59416968 24.4504127  26.48052369 28.52503096 30.63871813 32.83369895
 35.07911811 37.35674991]
[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0]
[12.06383479 12.06383479]
0


In [ ]:
np.savez("flux_truncate.npz",flux_file)
np.savez("par_truncate.npz",par_file)
np.savez("cross_truncate.npz",cross_file)

In [ ]:
flux_files=np.load("flux_truncate.npz")['arr_0']
par_files=np.load("par_truncate.npz")['arr_0']
cross_files=np.load("cross_truncate.npz")['arr_0']

In [ ]:
fig,ax=plt.subplots(3,3,figsize=(6,6))
c=['Reds','Blues','Greens']
for i in range(3):
    ax[0,i].pcolormesh(omega_d[:],nr,(np.transpose(flux_files[i,:,:])),cmap=c[i])
    ax[1,i].pcolormesh(omega_d[:],nr,(np.transpose(par_files[i,:,:])),cmap=c[i])
    ax[2,i].pcolormesh(omega_d[:],nr,(np.transpose(cross_files[i,:,:])),cmap=c[i])

In [ ]:
#we need to run the truncation simulations here after everything else is done: it should take 140hrs=6 days!